In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/Titanic_Dataset/train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
cols_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']

In [6]:
data = data.drop(cols_drop, axis = 1)

In [7]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
lab_enc = LabelEncoder()

In [10]:
data['Sex'] = lab_enc.fit_transform(data['Sex'])

In [11]:
data['Embarked'] = lab_enc.fit_transform(data['Embarked'])

In [12]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [14]:
data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [15]:
data = data.fillna(data['Age'].mean())

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int32  
dtypes: float64(2), int32(2), int64(4)
memory usage: 48.9 KB


In [17]:
input_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
output_col = ['Survived']

X_data = data[input_cols]
Y_data = data[output_col]

In [18]:
print(X_data.shape, Y_data.shape)

(891, 7) (891, 1)


In [46]:
def entropy(col):
    favour = np.unique(col, return_counts = True)
    T = col.shape[0]
    ent_py = 0
    
    for i in favour[1]:
        prob = i/T
        ent_py += -1*(prob * np.log2(prob))
    return ent_py

In [47]:
a = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,0])
print(len(list(np.unique(a, return_counts=True))))
print(np.unique(a, return_counts=True))

2
(array([0, 1]), array([ 9, 16], dtype=int64))


In [48]:
def divide_data(X_data, feat, threshold):
    X_right = pd.DataFrame([], columns = X_data.columns)
    X_left = pd.DataFrame([], columns = X_data.columns)
    
    for i in range(X_data.shape[0]):
        value = X_data[feat].loc[i]
        
        if value >= threshold:
            X_right = pd.concat([X_right, X_data.iloc[i: i + 1]])
        else:
            X_left = pd.concat([X_left, X_data.iloc[i: i + 1]])
    
    return X_left, X_right

In [49]:
left, right = divide_data(X_data[:10], 'Sex', 0.5)

In [50]:
print(left.shape, right.shape)

(5, 7) (5, 7)


In [51]:
print(left)

  Pclass Sex   Age SibSp Parch     Fare Embarked
1      1   0  38.0     1     0  71.2833        0
2      3   0  26.0     0     0   7.9250        2
3      1   0  35.0     1     0  53.1000        2
8      3   0  27.0     0     2  11.1333        2
9      2   0  14.0     1     0  30.0708        0


In [52]:
print(right)

  Pclass Sex        Age SibSp Parch     Fare Embarked
0      3   1  22.000000     1     0   7.2500        2
4      3   1  35.000000     0     0   8.0500        2
5      3   1  29.699118     0     0   8.4583        1
6      1   1  54.000000     0     0  51.8625        2
7      3   1   2.000000     3     1  21.0750        2


In [53]:
def info_gain(X_data, feat, threshold):
    left, right = divide_data(X_data, feat, threshold)
    
    prob_l = left.shape[0]/X_data.shape[0]
    prob_r = right.shape[0]/X_data.shape[0]
    
    i_gain = entropy(X_data.Survived)  - (prob_l * entropy(left.Survived)+
                                         prob_r * entropy(right.Survived))
    return i_gain

In [54]:
for i in X_data.columns:
    print(i, info_gain(data, i, data[i].mean()))

Pclass 0.07579362743608165
Sex 0.2176601066606142
Age 0.001158644038169343
SibSp 0.009584541813400071
Parch 0.015380754493137694
Fare 0.042140692838995464
Embarked 0.015909401384176403


In [55]:
print(pd.__version__)

1.3.0


In [68]:
class Decision_Tree:
    
    def __init__(self, curr_depth = 0, max_depth = 10):
        self.left = None
        self.right = None
        self.feat = None
        self.value = None
        self.depth = curr_depth
        self.max_depth = max_depth
        self.target = None
        
    def train(self, X_data):
        f = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
        i_gain = []
        
        for i in f:
            i_gain.append(info_gain(X_data, i, X_data[i].mean()))
        
        self.feat = f[np.argmax(i_gain)]
        
        self.value = X_data[self.feat].mean()
        print('Making a tree with feature:', self.feat, 'Depth:', self.depth)
        left, right = divide_data(X_data, self.feat, self.value)
        left = left.reset_index(drop = True)
        right = right.reset_index(drop = True)
        
        if left.shape[0] == 0 or right.shape[0] == 0:
            if X_data.Survived.mean() >= 0.5:
                self.target = 'Survived'
            else:
                self.target = 'Dead'
            return
        
        if self.depth >= self.max_depth:
            if X_data.Survived.mean() >= 0.5:
                self.target = 'Survived'
            else:
                self.target = 'Dead'
            return
        
        if X_data.Survived.mean() >= 0.5:
            self.target = 'Survived'
        else:
            self.target = 'Dead'
        
        self.left = Decision_Tree(curr_depth = self.depth + 1, max_depth = self.max_depth)
        self.left.train(left)
        
        self.right = Decision_Tree(curr_depth = self.depth + 1, max_depth = self.max_depth)
        self.right.train(right)
        
        return
    
    def predict(self, test):
        if test[self.feat] > self.value:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [73]:
split = 0.8

train_data = data[:int(split * data.shape[0])]
test_data = data[int(split * data.shape[0]):]

test_data = test_data.reset_index(drop = True)

In [74]:
print(train_data.shape)

(712, 8)


In [75]:
dt = Decision_Tree()
dt.train(train_data)

Making a tree with feature: Sex Depth: 0
Making a tree with feature: Pclass Depth: 1
Making a tree with feature: Embarked Depth: 2
Making a tree with feature: Age Depth: 3
Making a tree with feature: Pclass Depth: 4
Making a tree with feature: Pclass Depth: 5
Making a tree with feature: Pclass Depth: 5
Making a tree with feature: Age Depth: 4
Making a tree with feature: Pclass Depth: 5
Making a tree with feature: SibSp Depth: 5
Making a tree with feature: Fare Depth: 6
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 6
Making a tree with feature: Age Depth: 3
Making a tree with feature: SibSp Depth: 4
Making a tree with feature: Age Depth: 5
Making a tree with feature: Pclass Depth: 6
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Age Depth: 6
Making a tree with feature: Age Depth: 7
Making a tree with feature: Pclass Depth: 8
Making a tr

Making a tree with feature: SibSp Depth: 7
Making a tree with feature: Fare Depth: 8
Making a tree with feature: Age Depth: 9
Making a tree with feature: Pclass Depth: 10
Making a tree with feature: Fare Depth: 10
Making a tree with feature: Age Depth: 9
Making a tree with feature: Fare Depth: 10
Making a tree with feature: Fare Depth: 10
Making a tree with feature: Fare Depth: 8
Making a tree with feature: Pclass Depth: 9
Making a tree with feature: Age Depth: 9
Making a tree with feature: Pclass Depth: 10
Making a tree with feature: Pclass Depth: 10
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Age Depth: 8
Making a tree with feature: Age Depth: 9
Making a tree with feature: Pclass Depth: 10
Making a tree with feature: Fare Depth: 10
Making a tree with feature: Pclass Depth: 9
Making a tree with feature: Pclass Depth: 8
Making a tree with feature: SibSp Depth: 6
Making a tree with feature: Fare Depth: 7
Making a tree with feature: Pclass Depth: 8
Making a tr

Making a tree with feature: Pclass Depth: 6
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 8
Making a tree with feature: Pclass Depth: 8
Making a tree with feature: SibSp Depth: 3
Making a tree with feature: Embarked Depth: 4
Making a tree with feature: Pclass Depth: 5
Making a tree with feature: Pclass Depth: 6
Making a tree with feature: Pclass Depth: 6
Making a tree with feature: Pclass Depth: 5
Making a tree with feature: SibSp Depth: 6
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Pclass Depth: 8
Making a tree with feature: Age Depth: 8
Making a tree with feature: Pclass Depth: 9
Making a tree with feature: Pclass Depth: 9
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Parch Depth: 6
Making a tree with feature: Pclass Depth: 7
Making a tree with feature: Age Depth: 7
Making a tree with feature: Age Depth: 8
Making a tree with feature: Pclass Depth: 